In [0]:
spark.sql("USE CATALOG adb_cliente_savana_prd")
meu_nome = "elenir"

bronze_transacoes = spark.table(f"elenir_bronze.transacoes")


In [0]:
bronze_transacoes.display()

In [0]:
from pyspark.sql.functions import col, lower, trim, regexp_replace

silver_transacoes = bronze_transacoes \
    .withColumn("id", col("_id.oid")) \
    .withColumn("id", regexp_replace(col("id"), "[-_:\\\\s'\\\\/().$!@]", "")) \
    .withColumn("Meio_de_Pgmto", lower(trim(col("Meio_de_Pgmto")))) \
    .withColumn("Tipo", lower(trim(col("Tipo")))) \
    .withColumn("oid", col("_id.oid")) \
    .drop("_id") \
    .dropna(subset=["ID_Cliente", "ID_Transacao", "Valor", "Meio_de_Pgmto", "Tipo", "id"])



In [0]:
bronze_transacoes.printSchema()


In [0]:
# Exemplo de padronização simples (pode expandir com mais regras)
silver_transacoes = silver_transacoes.replace({
    "credito": "cartao de crédito",
    "debito": "cartao de débito",
    "pix": "pix"
}, subset=["Meio_de_Pgmto"])

In [0]:
silver_transacoes.write.mode("overwrite").saveAsTable(f"elenir_silver.transacoes_tratadas")


In [0]:
display(silver_transacoes)

In [0]:
# SILVER - Enriquecimento
bronze_clientes = spark.table("adb_cliente_savana_prd.elenir_bronze.clientes")
bronze_transacoes = spark.table("adb_cliente_savana_prd.elenir_bronze.transacoes")

# Join e limpeza
transacoes_enriquecidas = bronze_transacoes.join(
    bronze_clientes,
    bronze_transacoes["cliente_id"] == bronze_clientes["id"],
    "left"
).drop("id")  # Remove duplicidade se necessário

# Salvar na camada Silver
transacoes_enriquecidas.write.mode("overwrite").saveAsTable("savana.elenir.silver.transacoes_enriquecidas")
